In [1]:
import pandas as pd
import utils
from plotly.io import show

from skfolio import Population, RiskMeasure, Portfolio
from skfolio.model_selection import cross_val_predict, CombinatorialPurgedCV
from skfolio.optimization import (
    MeanRisk,
    ObjectiveFunction,
    EqualWeighted,
    InverseVolatility,
)

# Fetch Data


In [2]:
tickers = [
   "GPIL",
   "CUMMINSIND",
   "HCLTECH",
   "SUNPHARMA",
   "TATAPOWER",
   "ADANIGREEN",
   "HDFCAMC",
   "LTTS",
   "NMDC",
   "TEGA",
]

In [3]:
ret = utils.get_multiple_returns(tickers)
ret

,GPIL,CUMMINSIND,HCLTECH,SUNPHARMA,TATAPOWER,ADANIGREEN,HDFCAMC,LTTS,NMDC,TEGA
Date,,,,,,,,,,
2021-12-14,-0.014434,-0.030285,0.002804,-0.008589,-0.005212,0.036365,-0.010621,-0.007021,0.021152,-0.091729
2021-12-15,-0.020278,-0.027288,-0.011571,0.025197,-0.005239,0.002019,-0.012243,-0.003595,-0.005770,-0.065130
2021-12-16,-0.028172,0.001333,0.009444,-0.013740,-0.026114,-0.025708,-0.007734,0.019239,-0.010156,0.022384
2021-12-17,-0.018931,-0.024554,0.010045,0.005887,-0.027490,0.007845,-0.016327,-0.011742,-0.010993,-0.025067
2021-12-20,-0.033166,-0.023316,-0.009817,-0.006502,-0.031047,-0.030957,-0.035176,-0.074879,-0.032975,-0.074125
...,...,...,...,...,...,...,...,...,...,...
2024-04-05,0.020214,0.000743,0.003637,-0.006391,0.001207,0.005870,-0.006663,-0.003869,0.005074,-0.016958
2024-04-08,-0.014547,-0.014446,-0.003365,-0.005780,0.004221,0.009017,-0.003905,-0.018083,0.028453,0.013438
2024-04-09,0.011823,-0.016682,0.000292,0.001782,0.032545,-0.001511,0.011898,-0.012234,0.017403,-0.005475


In [4]:
nse = utils.get_returns("^NSEI", index=True)
nse.index = nse["Date"]
nse = nse.drop(columns=["Date"]).loc[ret.index[0] :]
nse

,^NSEI
Date,
2021-12-14,-0.002496
2021-12-15,-0.005974
2021-12-16,0.001568
2021-12-17,-0.015259
2021-12-20,-0.021843
...,...
2024-04-05,-0.000042
2024-04-08,0.006778
2024-04-09,-0.001039


# Models


### Sharpe Ratio Maximization


In [5]:
sharpe_model = MeanRisk(
    risk_measure=RiskMeasure.VARIANCE,
    objective_function=ObjectiveFunction.MAXIMIZE_RATIO,
    portfolio_params=dict(name="Max Sharpe"),
)
sharpe_pred = sharpe_model.fit_predict(ret)
sharpe_pred.annualized_sharpe_ratio

2.660935401986078

### Minimum Variance


In [6]:
inv_var_model = InverseVolatility(portfolio_params=dict(name="Minimum Variance"))
inv_var_pred = inv_var_model.fit_predict(ret)
inv_var_pred.annualized_sharpe_ratio

1.9725716002192832

### Equally Weighted


In [7]:
equ_model = EqualWeighted(portfolio_params=dict(name="Equal Weight"))
equ_pred = equ_model.fit_predict(ret)
equ_pred.annualized_sharpe_ratio

1.860637185226531

### NIFTY50 Benchmark


In [8]:
benchmark = EqualWeighted(portfolio_params=dict(name="NIFTY50"))
bench_pred = benchmark.fit_predict(nse)
bench_pred.annualized_sharpe_ratio

0.8908399360807827

### Clustering Optimization


In [9]:
import skfolio.optimization as opt

inner_estimator = MeanRisk(
    objective_function=ObjectiveFunction.MAXIMIZE_RATIO,
    risk_measure=RiskMeasure.VARIANCE,
)
outer_estimator = opt.RiskBudgeting(risk_measure=RiskMeasure.CVAR)

cluster_model = opt.NestedClustersOptimization(
    inner_estimator=inner_estimator,
    outer_estimator=outer_estimator,
    n_jobs=-1,
    portfolio_params=dict(name="NCO-1"),
)
cluster_pred = cluster_model.fit_predict(ret)
cluster_pred.annualized_sharpe_ratio

2.225931993533894

### Rolling Window Optimization


In [10]:
rolling_pred = utils.rolling_window_portfolio(ret, 5)

In [11]:
population = Population(
    [cluster_pred, sharpe_pred, inv_var_pred, equ_pred, rolling_pred, bench_pred]
)
population.set_portfolio_params(compounded=True)
population.plot_composition()

In [12]:
population.plot_cumulative_returns()

In [13]:
summary = population.summary()
summary[summary.index.str.contains("Annualized")]

,NCO-1,Max Sharpe,Minimum Variance,Equal Weight,Rolling Window,NIFTY50
Annualized Mean,42.49%,45.35%,33.98%,34.81%,41.83%,12.35%
Annualized Variance,3.64%,2.90%,2.97%,3.50%,4.74%,1.92%
Annualized Semi-Variance,2.01%,1.44%,1.66%,1.98%,2.54%,1.04%
Annualized Standard Deviation,19.09%,17.04%,17.23%,18.71%,21.77%,13.86%
Annualized Semi-Deviation,14.19%,12.01%,12.90%,14.07%,15.94%,10.21%
Annualized Sharpe Ratio,2.23,2.66,1.97,1.86,1.92,0.89
Annualized Sortino Ratio,2.99,3.78,2.63,2.47,2.62,1.21


In [14]:
population.composition() * 1_00_000

,NCO-1,Max Sharpe,Minimum Variance,Equal Weight,Rolling Window,NIFTY50
asset,,,,,,
SUNPHARMA,26252.053809,40762.670793,15618.329080,10000.0,20636.999826,0.0
CUMMINSIND,24238.268864,36836.312340,11747.973013,10000.0,7598.788166,0.0
ADANIGREEN,15950.546668,0.000000,5425.363084,10000.0,3978.841054,0.0
NMDC,12674.824943,10003.083703,9070.879113,10000.0,2696.082689,0.0
GPIL,8384.751454,7413.404010,6842.840713,10000.0,31998.535853,0.0
TATAPOWER,6548.675673,0.000000,9681.357468,10000.0,16034.980436,0.0
TEGA,4357.354660,4984.528974,8176.993881,10000.0,688.601011,0.0
HCLTECH,1593.523912,0.000000,13242.041375,10000.0,9050.264954,0.0
HDFCAMC,0.000000,0.000000,10582.538683,10000.0,3866.824990,0.0
